# OCR
This project implements OCR for the first 10 pages of a general PDF. This program will be compatiable with SQL.

### Things to Note
- to use a juypter notebook in vs code through uv, you must do these things:
    - uv init the project
    - create a venv
    - uv add --dev ipykernel
    - in vs code when selecting a kernel for the notebook, select pyhton then your venv
- Using `easyocr`
    - in order to read pdf documents, I've had to using the package pdf2image which requires poppler binary files which are attached in this project
    - this introduces system a system depency with the bin files
    - also I've been experiencing some crashing on my computer due to the model running on my cpu I think
    - I have a gpu though I'm not exactly sure how to get it to run on there
    - it would be most ideal if I could use google colabs gpus (I already have an account with them)
    - using easyocr automatically uses a cpu version of torch, even tried to uninstall easyocr and download a cuda 12.6 torch version
      but "uv add easyocr" deletes gpu version and reinstalls cpu version

### Next Steps
- increase computation speed (switch to gpu)
- visualize which words are read on pdf (box detected letters)
- add compatability with SQL

In [1]:
import sys
# check env
print(sys.executable)

c:\Users\dsbor\OneDrive\Desktop\Personal\Davis-PDF-Extraction\.venv\Scripts\python.exe


In [1]:
# check if gpu is there
import torch
print(torch.cuda.is_available())

True


In [ ]:
# testing easyocr
import easyocr

reader = easyocr.Reader(['en'], gpu=True)
result = reader.readtext('good_ex.jpg')

for (bbox, text, prob) in result:
    print(f"Text: {text}, Probability: {prob:.4f}")

In [1]:
# testing pdf to image conversion
from pdf2image import convert_from_path
import easyocr
import numpy as np

pages = convert_from_path('HW3-1.pdf', poppler_path=r'poppler_bin', first_page=1, last_page=4)

reader = easyocr.Reader(['en'], gpu=True)

for i, page_image in enumerate(pages, start=1):
    # convert PIL image to np array
    img_np = np.array(page_image)

    result = reader.readtext(img_np)

    print(f"\n--- Text on page {i} ---")
    for bbox, text, prob in result:
        # print(f'Text: {text}, Probability: {prob:.4f}')
        print(f"{text}")

Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.
c:\Users\dsbor\OneDrive\Desktop\Personal\Davis-PDF-Extraction\.venv\Lib\site-packages\torch\utils\data\dataloader.py:775: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  super().__init__(loader)



--- Text on page 1 ---
Homework 3
ENGR 4230: Power Systems
Problem 1
2.15
Let a voltage source v(t)
= 4 cos(ot + 609) be connected to an imped-
ance Z = 2[300 0 (a) Given the operating frequency to be 60
determine the expressions for the current and instantaneous power
delivered by the source as functions of time. (b) Plot these functions
along with v(t)
on
a
single graph for comparison. (c) Find the fre-
quency and average value of the instantaneous power:
Pro tip-
Desmos (https /LWWWdesmos_com/calculator) is a great resource for plotting:
They even have a phasor plotter! (https:/ IWW.desmos com/calculator/niadSyjthy)
Hz,

--- Text on page 2 ---
Homework 3
ENGR 4230: Power Systems
Problem 2
2.21
An
industrial
plant
consisting   primarily
of
induction
motor
loads
absorbs
500
kW
at
0.6  power
factor   lagging     (a)
Com-
pute the required kVA rating of
a shunt
capacitor to improve the
power factor to 0.9 lagging (b) Calculate the resulting power fac-
tor if
a
synchronous
motor rated
a

In [ ]:
# putting it all together and boxing and saving what is reconginzed
from pdf2image import convert_from_path
import easyocr
import numpy as np
import cv2

pages = convert_from_path('example.pdf', poppler_path=r'poppler_bin', first_page=1, last_page=10)

reader = easyocr.Reader(['en'], gpu=True)

for i, page_image in enumerate(pages, start=1):
    
    img_np = np.array(page_image)
    img_cv = cv2.cvtColor(img_np, cv2.COLOR_RGB2BGR)

    result = reader.readtext(img_np)

    for bbox, text, prob in result:
        
        pts = np.array(bbox, dtype=np.int32)
        cv2.polylines(img_cv, [pts], isClosed=True, color=(0, 255, 0), thickness=2)

        top_left = tuple(pts[0])
        cv2.putText(img_cv, text, (top_left[0], top_left[1]-5),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)

    cv2.imwrite(f'page_{i}_boxed.png', img_cv)

    print(f"Page {i} processed, saved as page_{i}_boxed.png")
